In [39]:
import pandas as pd
import urllib.request, json
from datetime import datetime as dt
import time
from datetime import timedelta, date
import re
import pytz
import folium


In [ ]:
class weather_details():
    def __init__(self, lat, long):
        start = time.time()
        self.lat = lat
        self.long = long
        self.url = "https://api.weather.gov/points/" + str(lat) + ',' + str(long)
        self.data = self.return_json(self.url)

        self.forecastOffice = self.data['properties']['cwa']

        self.forecast()
        #assuming that elevations is always in meters
        self.elevation = self.forecast_detailed['properties']['elevation']['value']
        self.elevation_unit = self.forecast_detailed['properties']['elevation']['unitCode'].split(':')[1]

        self.forecast_hourly()
        self.properties()
    
    def forecast(self):
        self.forecast_url = self.data['properties']['forecast']
        
        forecast_detailed = self.return_json(self.forecast_url)  
        self.forecast_detailed = forecast_detailed
        self.forecast = {}
        self.forecast['coordinates'] = [item[::-1] for item in forecast_detailed['geometry']['coordinates'][0]]
        
        #forecast is provided as a list of dictionaries which are converted to columns of a dataframe here 
        df = pd.DataFrame(self.forecast_detailed['properties']['periods'])        
        df.drop(['number', 'endTime', 'temperatureTrend'], axis = 1, inplace = True)
        df['startTime'] = pd.to_datetime(df['startTime'])
        #convert date time to AM PM format for easier consumption
        df['startTime'] = df['startTime'].apply(lambda x : x.strftime('%Y-%m-%d %I:%M:%S %p'))
        #extract wind speed mins and max
        df['windSpeed_min'] = df['windSpeed'].apply(lambda x: int(x.replace(' mph', '').split(' to ')[0]))
        df['windSpeed_max']  = df['windSpeed'].apply(lambda x: int(x.replace(' mph', '').split(' to ')[-1]))
        df['probabilityOfPrecipitation'] = df['probabilityOfPrecipitation'].apply( lambda x: x['value'] / 100 if x['value'] is not None else 0 )
        #df.drop(['number', 'endTime', 'temperatureTrend', 'windSpeed'], axis = 1, inplace = True)
        df = df.set_index('startTime')

        self.forecast['fullforecast_df'] = df
        #assuming temperature unit is always F
        self.forecast['TempForecast_df'] = df[['temperature']]
        self.forecast['Probability of precipitation'] = df[['probabilityOfPrecipitation']]
        self.forecast['wind'] = df[['windSpeed_min', 'windSpeed_max', 'windDirection']]

    def forecast_hourly(self):
        self.forecastHourly_url = self.data['properties']['forecastHourly']
        forecastHourly_detailed = self.return_json(self.forecastHourly_url)
        self.forecastHourly_detailed = forecastHourly_detailed

    def properties(self):
        #defining properties such as forecast generator type, forecast generating date etc...
        self.forecast_specifications = {'forecast' : {},
                                        'Hourlyforecast' : {}}
        properties = self.forecast_detailed['properties']
        self.forecast_specifications['forecast']['forecastGenerator'] = properties['forecastGenerator']
        self.forecast_specifications['forecast']['generatedAt'] = dt.fromisoformat(properties['generatedAt'])
        self.forecast_specifications['forecast']['updateTime'] = dt.fromisoformat(properties['updateTime'])

        validtimes_str = properties['validTimes']
        self.forecast_specifications['forecast']['validTime_start'] = dt.fromisoformat( validtimes_str.split('/')[0])
        #extract how many days and hour it is valid for
        [d, h] = re.findall( r'\d+', validtimes_str.split('/')[1])
        self.forecast_specifications['forecast']['validTime_ends'] = self.forecast_specifications['forecast']['validTime_start'] + timedelta(days = int(d), hours = int(h))

    def return_json(self, url_api):
        with urllib.request.urlopen(url_api) as url:
            data = json.load(url)
            return data
    #coordinates in the format of [[lat, lon]...]
    def polygon_to_map(self, coordinates = -1, zoom_start = 10):
        if coordinates == -1:
            coordinates = self.forecast['coordinates'] 
        lat_mean = sum([item[0] for item in coordinates])/len(coordinates)
        long_mean = sum([item[1] for item in coordinates ])/len(coordinates)
        m = folium.Map(location=[lat_mean, long_mean], zoom_start=zoom_start)
        folium.Polygon(
            locations = coordinates,  # Flip lon/lat to lat/lon
            color='blue',
            fill=True,
            fill_color='cyan',
            fill_opacity=0.5
        ).add_to(m)
        return(m)

def return_json(url_api):
    with urllib.request.urlopen(url_api) as url:
        data = json.load(url)
        return data
def weather_api_coordinates(lat, long):
    url_api = "https://api.weather.gov/points/" + str(lat) + ',' + str(long)
    return(return_json(url_api))

def polygon_to_map(coordinates): #coordinates in the format of [[lat, lon]...]
    m = folium.Map(location=coordinates[0], zoom_start=13)
    folium.Polygon(
        locations = coordinates,  # Flip lon/lat to lat/lon
        color='blue',
        fill=True,
        fill_color='cyan',
        fill_opacity=0.5
    ).add_to(m)
    return(m)



lat = 47.675908
long = -122.208008

lat = 46.189070
long = -122.184252

#data = weather_api_coordinates(lat, long)

In [46]:
test = weather_details(lat, long)
#test.polygon_to_map()

https://api.weather.gov/points/46.18907,-122.184252
0.0
19.836196422576904
0.0
4.613446474075317


In [34]:
m = test.polygon_to_map(zoom_start = 15)
m

In [30]:
test.forecast['coordinates']

[[46.190432, -122.1962409],
 [46.1696659, -122.19014320000001],
 [46.1738883, -122.1601332],
 [46.1946546, -122.1662245],
 [46.190432, -122.1962409]]

In [26]:
lat_mean = sum([item[0] for item in test.forecast['coordinates']])/len(test.forecast['coordinates'])
long_mean = sum([item[1] for item in test.forecast['coordinates']])/len(test.forecast['coordinates'])
long_mean


-122.18179654000001

In [23]:
lat_mean

46.18381456

In [24]:
long_mean

-122.18179654